In [9]:
import numpy as np

# Original Lasso implementation (as you provided)
def original_lasso(x, y, lmd, tol=1e-4, beta=None, intercept=True):
    def _soft_threshold(b, gamma):
        return np.sign(b) * np.maximum(np.abs(b) - gamma, 0)

    def _gamma(n, lmd, x):
        gammas = np.zeros(x.shape[1])
        for i in range(x.shape[1]):
            gammas[i] = n * lmd / x[:, i].dot(x[:, i])
        return gammas

    def _lasso_coordinate_descent(**params):
        y_mean = params['y_mean']
        x_col_mean = params['x_col_mean']
        XTX = params['XTX']
        YTX = params['YTX']
        X_j_squared = params['X_j_squared']
        n = params['n']
        gammas = params['gammas']
        beta = params['beta']
        intercept = params['intercept']

        if intercept:
            b0 = y_mean - x_col_mean.dot(beta)
        else:
            b0 = 0
        abs_change = 0

        for j in range(len(beta)):
            gamma_j = gammas[j]

            z_j_dot_x_j = YTX[j] - x_col_mean[j] * n * b0 - sum([XTX[j, k] * beta[k] for k in range(len(beta)) if k != j])

            beta_hat_j = z_j_dot_x_j / X_j_squared[j]
            new_beta_j = _soft_threshold(beta_hat_j, gamma_j)
            abs_change = np.abs(beta[j] - new_beta_j) if np.abs(beta[j] - new_beta_j) > abs_change else abs_change
            beta[j] = new_beta_j

        return b0, beta, abs_change

    epsilon = np.inf
    n, p = x.shape
    y_mean = np.mean(y)
    x_col_mean = np.mean(x, axis=0)
    gammas = _gamma(n, lmd, x)

    if intercept:
        beta = np.zeros(p) if beta is None else beta[1:]
        b0 = 0 if beta is None else beta[0]
    else:
        beta = np.zeros(p) if beta is None else beta
        b0 = 0

    XTX = x[:].T.dot(x[:])
    YTX = y[:].T.dot(x[:])

    X_j_squared = np.zeros(p)
    for j in range(p):
        X_j_squared[j] = x[:, j].dot(x[:, j])

    params = {
        'y_mean': y_mean,
        'x_col_mean': x_col_mean,
        'XTX': XTX,
        'YTX': YTX,
        'X_j_squared': X_j_squared,
        'n': n,
        'gammas': gammas,
        'beta': beta,
        'intercept': intercept
    }

    while epsilon > tol:
        b0, beta, epsilon = _lasso_coordinate_descent(**params)

    if intercept:
        beta = np.insert(beta, 0, b0)
    return beta

# Refactored Lasso implementation
def refactored_lasso(X, y, lambda_, tol=1e-4, beta=None, fit_intercept=True):
    n, p = X.shape
    y_mean = np.mean(y)
    X_col_mean = np.mean(X, axis=0)
    
    if fit_intercept:
        beta = np.zeros(p) if beta is None else beta[1:]
        intercept = 0 if beta is None else beta[0]
    else:
        beta = np.zeros(p) if beta is None else beta
        intercept = 0

    XTX = X.T @ X
    YTX = X.T @ y
    X_j_squared = np.array([X[:, j].T @ X[:, j] for j in range(p)])

    gamma = lambda_ * n / X_j_squared

    def soft_threshold(b, threshold):
        return np.sign(b) * np.maximum(np.abs(b) - threshold, 0)

    change = np.inf
    while change > tol:
        change = 0
        if fit_intercept:
            intercept = y_mean - np.dot(X_col_mean, beta)
        
        for j in range(p):
            residual = YTX[j] - np.dot(XTX[j, :], beta) + XTX[j, j] * beta[j]
            beta_hat_j = (residual - n * intercept * X_col_mean[j]) / X_j_squared[j]
            new_beta_j = soft_threshold(beta_hat_j, gamma[j])
            change = max(change, np.abs(beta[j] - new_beta_j))
            beta[j] = new_beta_j

    if fit_intercept:
        return np.insert(beta, 0, intercept)
    else:
        return beta

# Test function to compare both implementations
def test_lasso_implementations():
    # Generate random data for testing
    np.random.seed(42)
    X = np.random.randn(100, 1424)
    y = np.random.randn(100)
    lambda_ = 0.4
    tol = 1e-10

    # Run both the original and refactored implementations
    beta_original = original_lasso(X, y, lambda_, tol=tol)
    beta_refactored = refactored_lasso(X, y, lambda_, tol=tol)

    # Compare the results (allowing for small numerical differences)
    assert np.allclose(beta_original, beta_refactored, atol=1e-5), "Results differ between the original and refactored implementation."

    print("Test passed! Both implementations produce the same result.")

# Run the test
test_lasso_implementations()


Test passed! Both implementations produce the same result.
